In [ ]:
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import plotly.express as px
from dotenv import load_dotenv
import pandas as pd
import base64
import os
import numpy as np
import matplotlib.pyplot as plt

from AnimalShelter import AnimalShelter  # My Crud Class

# Get environment variables from .env file
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')

# Initialize database connection
db = AnimalShelter(username=username, password=password)

# Load data from MongoDB
df = pd.DataFrame.from_records(db.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Initialize Dash app
app = Dash(__name__)

# Add the logo image
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Create the dashboard layout
app.layout = html.Div([
    # Header with title and logo
    html.Div([
        html.Center(html.H1('Grazioso Salvare Dashboard - Devin Wheeler')),
        html.Center(html.Img(
            src=f'data:image/png;base64,{encoded_image}',
            style={'height': '64px', 'width': 'auto'}
        ))
    ], style={'padding': '20px', 'backgroundColor': '#f9f9f9'}),

    html.Hr(),

    # Filter controls
    html.Div([
        dcc.Dropdown(
            id='animal-type-dropdown',
            options=[{'label': atype, 'value': atype} for atype in df['animal_type'].unique()],
            placeholder='Filter by Animal Type',
            multi=True
        ),
        html.Br(),
        html.Div(id='rescue-category-container', children=[
            html.Label("Select Rescue Category:"),
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                    {'label': 'Reset', 'value': 'Reset'}
                ],
                value='Reset',
                labelStyle={'display': 'inline-block', 'margin-right': '15px'}
            )
        ]),
        html.Div(id='conditional-break'),
        dcc.Dropdown(
            id='outcome-type-dropdown',
            options=[{'label': outcome, 'value': outcome} for outcome in df['outcome_type'].unique()],
            placeholder='Filter by Outcome Type',
            multi=True
        ),
        html.Br(),
        dcc.Dropdown(
            id='sort-by-dropdown',
            options=[
                {'label': 'Name', 'value': 'name'},
                {'label': 'Age (weeks)', 'value': 'age_upon_outcome_in_weeks'},
                {'label': 'Breed', 'value': 'breed'}
            ],
            placeholder='Sort by...',
            value='name'
        )
    ], style={'padding': '10px 20px'}),

    html.Hr(),

    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '200px'
        },
        style_data={'whiteSpace': 'normal', 'height': 'auto'},
        row_selectable='single',
        selected_rows=[],
        sort_action='native',
        filter_action='native'
    ),

    html.Br(),
    html.Hr(),

    # Chart and Map side-by-side
    html.Div([
        html.Div(id='graph-id', className='six columns', style={'width': '50%', 'display': 'inline-block'}),
        html.Div(id='map-id', className='six columns', style={'width': '50%', 'display': 'inline-block'})
    ], style={'padding': '20px'})
])


####################################
# Callback: Toggle Rescue Category Visibility
####################################

# Function to toggle rescue category visibility
@app.callback(
    Output('rescue-category-container', 'style'),
    Output('filter-type', 'value'),
    Input('animal-type-dropdown', 'value')
)
def toggle_rescue_visibility(selected_animals):
    if selected_animals == ['Dog']:
        return {'display': 'block'}, 'Reset'
    return {'display': 'none'}, 'Reset'

# Function to add a break if dog is selected
@app.callback(
    Output('conditional-break', 'children'),
    Input('animal-type-dropdown', 'value')
)
def show_break_if_dog_selected(selected_animals):
    if selected_animals == ['Dog']:
        return html.Br()
    return None

####################################
# Callback: Update Data Table
####################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value'),
     Input('animal-type-dropdown', 'value'),
     Input('outcome-type-dropdown', 'value'),
     Input('sort-by-dropdown', 'value')]
)
def update_dashboard(filter_type, animal_type, outcome_type, sort_by):
    query = {}

    # Apply rescue category filters
    if filter_type == 'Water':
        query.update({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        })
    elif filter_type == 'Mountain':
        query.update({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        })
    elif filter_type == 'Disaster':
        query.update({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        })

    # Apply new filters to the query
    if animal_type:
        query['animal_type'] = {"$in": animal_type}
    if outcome_type:
        query['outcome_type'] = {"$in": outcome_type}

    # Fetch filtered data
    df = pd.DataFrame.from_records(db.read(query))
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    if sort_by and sort_by in df.columns:
        df = df.sort_values(by=sort_by)
    
    return df.to_dict('records')

####################################
# Callback: Update Pie Chart
####################################

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if not viewData:
        return [html.Div("No data to display.", style={'textAlign': 'center', 'fontSize': 20, 'marginTop': 20})]

    df = pd.DataFrame(viewData)
    fig = px.pie(df, names='breed', title='Distribution of Animal Breeds') if not df.empty else px.pie(title='No Data Available')

    return [dcc.Graph(figure=fig)]


####################################
# Callback: Highlight Selected Column
####################################

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

####################################
# Callback: Update Map
####################################

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):

    #Add safety check for blank data
    if not viewData or not isinstance(viewData, list) or not index:
        return [html.Div("No data to display on the map.")]
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run the app
app.run(mode='external', debug=True, use_reloader=False)
